In [1]:
import os.path as op
import itertools

import pandas as pd
import numpy as np
from pymare.stats import fdr

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [101]:
roi_dict = {
    "vmPFC1": "vmPFC",
    "vmPFC2": "vmPFC",
    "vmPFC3": "vmPFC",
    "vmPFC4": "vmPFC",
    "vmPFC5": "vmPFC",
    "vmPFC6": "vmPFC",
    "insulaDlh": "Insula",
    "insulaPlh": "Insula",
    "insulaVlh": "Insula",
    "insulaDrh": "Insula",
    "insulaPrh": "Insula",
    "insulaVrh": "Insula",
    "hippocampus3solF1lh": "Hippocampus",
    "hippocampus3solF2lh": "Hippocampus",
    "hippocampus3solF3lh": "Hippocampus",
    "hippocampus3solF1rh": "Hippocampus",
    "hippocampus3solF2rh": "Hippocampus",
    "hippocampus3solF3rh": "Hippocampus",
    "striatumMatchCDlh": "Striatum",
    "striatumMatchCVlh": "Striatum",
    "striatumMatchDLlh": "Striatum",
    "striatumMatchDlh": "Striatum",
    "striatumMatchRlh": "Striatum",
    "striatumMatchVlh": "Striatum",
    "striatumMatchCDrh": "Striatum",
    "striatumMatchCVrh": "Striatum",
    "striatumMatchDLrh": "Striatum",
    "striatumMatchDrh": "Striatum",
    "striatumMatchRrh": "Striatum",
    "striatumMatchVrh": "Striatum",
    "amygdala1lh": "Amygdala",
    "amygdala2lh": "Amygdala",
    "amygdala3lh": "Amygdala",
    "amygdala1rh": "Amygdala",
    "amygdala2rh": "Amygdala",
    "amygdala3rh": "Amygdala",
}

seed_dict = {
    "vmPFC1": "R sgACC",
    "vmPFC2": "Anterior Medial",
    "vmPFC3": "Central Dorsal",
    "vmPFC4": "Posterior",
    "vmPFC5": "Central Ventral",
    "vmPFC6": "L sgACC",
    "insulaDlh": "L Dorsal Anterior",
    "insulaPlh": "L Posterior",
    "insulaVlh": "L Ventral Anterior",
    "insulaDrh": "R Dorsal Anterior",
    "insulaPrh": "R Posterior",
    "insulaVrh": "R Ventral Anterior",
    "hippocampus3solF1lh": "L Anterior",
    "hippocampus3solF2lh": "L Intermediate",
    "hippocampus3solF3lh": "L Posterior",
    "hippocampus3solF1rh": "R Anterior",
    "hippocampus3solF2rh": "R Intermediate",
    "hippocampus3solF3rh": "R Posterior",
    "striatumMatchCDlh": "L Caudal (Dorsal)",
    "striatumMatchCVlh": "L Caudal (Ventral)",
    "striatumMatchDLlh": "L Dorsolateral",
    "striatumMatchDlh": "L Dorsal",
    "striatumMatchRlh": "L Rostral",
    "striatumMatchVlh": "L Ventral",
    "striatumMatchCDrh": "R Caudal (Dorsal)",
    "striatumMatchCVrh": "R Caudal (Ventral)",
    "striatumMatchDLrh": "R Dorsolateral",
    "striatumMatchDrh": "R Dorsal",
    "striatumMatchRrh": "R Rostral",
    "striatumMatchVrh": "R Ventral",
    "amygdala1lh": "L Centromedial Nuclei",
    "amygdala2lh": "L Superficial Nuclei",
    "amygdala3lh": "L Laterobasal Nuclei",
    "amygdala1rh": "R Centromedial Nuclei",
    "amygdala2rh": "R Superficial Nuclei",
    "amygdala3rh": "R Laterobasal Nuclei",
}

dset_dict = {
    "ALC": "Alcohol (ALC) Dataset",
    "ATS": "Methamphetamine and Dexamphetamine (ATS) Dataset",
    "CANN": "Cannabis (CANN) Dataset",
    "COC": "Cocaine (COC) Dataset",
}

metric_dict = {
    "REHO": "Regional Homogeneity (ReHo)",
    "FALFF": "Fractional Amplitude of Low Frequency Fluctuations (fALFF)",
}

In [139]:
dsets = ["ALC", "ATS", "CANN", "COC"]
gsrs = ["-gsr", ""]
methods = ["-lm", "-mlm", "-combat"]
metrics = ["REHO", "FALFF"]
seeds = ["amygdala", "hippocampus", "insula", "striatum", "vmPFC"]

for dset, metric in itertools.product(dsets, metrics):
    if metric == "REHO":
        metric_lb = "ReHo"
    else:
        metric_lb = "fALFF"
    
    concat_column_nm = []
    concat_table_nm = []
    concat_table = []
    significant_arr = np.zeros(36)
    for gsr, method in itertools.product(gsrs, methods):
        table = pd.DataFrame()
        rois_seed_lst = []
        rois_lst = []
        seed_lst = []
        pvals = []
        corr_pvals = []
        estimates = []
        ses = []
        cils = []
        cihs = []
        prev_len = 0
        for seed_i, seed in enumerate(seeds):
            if seed == "vmPFC":
                hemispheres = [""]
            else:
                hemispheres = ["lh", "rh"]
            for hemis in hemispheres:
                if seed == "amygdala":
                    rois = [f"amygdala1{hemis}", f"amygdala2{hemis}", f"amygdala3{hemis}"]
                elif seed == "hippocampus":
                    rois = [
                        f"hippocampus3solF1{hemis}",
                        f"hippocampus3solF2{hemis}",
                        f"hippocampus3solF3{hemis}",
                    ]
                elif seed == "insula":
                    rois = [f"insulaD{hemis}", f"insulaP{hemis}", f"insulaV{hemis}"]
                elif seed == "striatum":
                    rois = [
                        f"striatumMatchCD{hemis}",
                        f"striatumMatchCV{hemis}",
                        f"striatumMatchDL{hemis}",
                        f"striatumMatchD{hemis}",
                        f"striatumMatchR{hemis}",
                        f"striatumMatchV{hemis}",
                    ]
                elif seed == "vmPFC":
                    rois = ["vmPFC1", "vmPFC2", "vmPFC3", "vmPFC4", "vmPFC5", "vmPFC6"]

                sub_pvals = []
                for roi_i, roi in enumerate(rois):
                    data_df = pd.read_csv(op.join("./indiv_tables", f"{dset}-{metric}-{roi}{gsr}{method}.csv"))
                    sel_row = data_df.loc[data_df["term"] == "groupcontrol"]
                    pval = sel_row["p.value"].values[0]
                    estimate = sel_row["estimate"].values[0]
                    se = sel_row["std.error"].values[0]

                    sub_pvals.append(pval)
                    estimates.append(estimate)
                    ses.append(se)
                    cils.append(estimate - 1.96 * se)
                    cihs.append(estimate + 1.96 * se)

                # Correct p-values
                sub_corr_pvals = fdr(np.array(sub_pvals))
                for pval_i, sub_corr_pval in enumerate(sub_corr_pvals):
                    significant_arr[prev_len + pval_i] += 1 if sub_corr_pval < 0.05 else 0
                prev_len += len(rois)

                pvals.append(sub_pvals)
                corr_pvals.append(sub_corr_pvals)
                rois_seed_lst.append([f"{roi_dict[roi]}: {seed_dict[roi]}:" for roi in rois])
                rois_lst.append([roi_dict[roi] for roi in rois])
                seed_lst.append([seed_dict[roi] for roi in rois])

        pvals = np.hstack(pvals)
        corr_pvals = np.hstack(corr_pvals)
        rois_seed_lst = np.hstack(rois_seed_lst)
        rois_lst = np.hstack(rois_lst)
        seed_lst = np.hstack(seed_lst)

        # Write values to table
        table["ROI: Sedd"] = rois_seed_lst
        table = table.set_index("ROI: Sedd")
        table["Estimate (SE)"] = [
            f"{est:.3f} ({se:.3f})" if est > .001 else f"$<$.001 ({se:.3f})"
            for est, se in zip(estimates, ses)
        ]
        table["Uncorrected p-value"] = [f"{pval:.3f}$^{{*}}$" if pval < 0.05 else f"{pval:.3f}" for pval in pvals]
        table["FDR-corrected p-value"] = [
            f"{corr_pval:.3f}$^{{*}}$" if corr_pval < 0.05 else f"{corr_pval:.3f}" 
            for corr_pval in corr_pvals
        ]
        table["95\% CI L"] = [f"{cil:.3f}" for cil in cils]
        table["95\% CI H"] = [f"{cih:.3f}" for cih in cihs]

        table_nm = f"{dset}-{metric}{gsr}{method}.tex"
        print(f"\input{{{table_nm}}}")

        row_arrays = [rois_lst, seed_lst]
        row_tuples = list(zip(*row_arrays))
        row = pd.MultiIndex.from_tuples(row_tuples)
        table.index = row
        table.index = combined_df.index.rename(["ROI", "Seed"])

        label = "GSR: True" if gsr == "-gsr" else "GSR: False"
        label += ", COMBAT" if method == "-combat" else ", LM" if method == "-lm" else ", MLM"

        caption = (f"Results of {metric_dict[metric]} Analysis for {dset_dict[dset]}. {label}", f"{dset}-{metric}-{gsr}{method}")
        with open(op.join("./indiv_tables", table_nm), 'w') as tf:
            tf.write(table.style.to_latex(
                caption=caption, 
                hrules=True, 
                multicol_align="c",
                multirow_align="t",
                column_format="llccccc",
            ))

        concat_table.append(table.values[:, (0,2)])
        concat_table_nm.append([label]*2)
        concat_column_nm.append(["Estimate (SE)", "p-value"])
    
    col_arrays = [np.hstack(concat_table_nm), np.hstack(concat_column_nm)]
    col_tuples = list(zip(*col_arrays))
    col = pd.MultiIndex.from_tuples(col_tuples)

    combined_df = pd.DataFrame(np.hstack(concat_table), columns=col)

    combined_df["ROI"] = rois_seed_lst
    combined_df = combined_df.set_index("ROI")
    combined_df.index = row
    combined_df.index = combined_df.index.rename(["ROI", "Seed"])
    
    combined_df["Significant"] = [f"{int(sig)}/6$^{{*}}$" if sig > 3 else f"{int(sig)}/6" for sig in significant_arr]

    combined_table_nm = f"{dset}-{metric}.tex"
    
    caption = (f"Results of {metric_dict[metric]} Analysis for {dset_dict[dset]}.", f"{dset}-{metric}")
    latex = combined_df.style.to_latex(
            caption=caption, 
            hrules=True, 
            position="h",
            multicol_align="c",
            multirow_align="t",
            column_format="llccccccccccccc",
    )
    # column_format=r"lp{2cm}p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}p{1.2cm}",
    
    # Add midrules to multicolumns
    midrule_str = r"\cmidrule(lr){3-4} \cmidrule(lr){5-6} \cmidrule(lr){7-8} \cmidrule(lr){9-10} \cmidrule(lr){11-12} \cmidrule(lr){13-14}"
    latex_lines = latex.splitlines()
    latex_lines.insert(5, midrule_str)
    latex = '\n'.join(latex_lines)

    with open(op.join("./indiv_tables", combined_table_nm), 'w') as tf:
        tf.write(latex)



\input{ALC-REHO-gsr-lm.tex}
\input{ALC-REHO-gsr-mlm.tex}
\input{ALC-REHO-gsr-combat.tex}
\input{ALC-REHO-lm.tex}
\input{ALC-REHO-mlm.tex}
\input{ALC-REHO-combat.tex}
\input{ALC-FALFF-gsr-lm.tex}
\input{ALC-FALFF-gsr-mlm.tex}
\input{ALC-FALFF-gsr-combat.tex}
\input{ALC-FALFF-lm.tex}
\input{ALC-FALFF-mlm.tex}
\input{ALC-FALFF-combat.tex}
\input{ATS-REHO-gsr-lm.tex}
\input{ATS-REHO-gsr-mlm.tex}
\input{ATS-REHO-gsr-combat.tex}
\input{ATS-REHO-lm.tex}
\input{ATS-REHO-mlm.tex}
\input{ATS-REHO-combat.tex}
\input{ATS-FALFF-gsr-lm.tex}
\input{ATS-FALFF-gsr-mlm.tex}
\input{ATS-FALFF-gsr-combat.tex}
\input{ATS-FALFF-lm.tex}
\input{ATS-FALFF-mlm.tex}
\input{ATS-FALFF-combat.tex}
\input{CANN-REHO-gsr-lm.tex}
\input{CANN-REHO-gsr-mlm.tex}
\input{CANN-REHO-gsr-combat.tex}
\input{CANN-REHO-lm.tex}
\input{CANN-REHO-mlm.tex}
\input{CANN-REHO-combat.tex}
\input{CANN-FALFF-gsr-lm.tex}
\input{CANN-FALFF-gsr-mlm.tex}
\input{CANN-FALFF-gsr-combat.tex}
\input{CANN-FALFF-lm.tex}
\input{CANN-FALFF-mlm.tex}
\inp

In [137]:
table

Estimate (SE) Uncorrected p-value  \
ROI         Seed                                                         
Amygdala    L Centromedial Nuclei  $<$.001 (0.024)               0.927   
            L Superficial Nuclei   $<$.001 (0.044)               0.869   
            L Laterobasal Nuclei   $<$.001 (0.029)               0.402   
            R Centromedial Nuclei  $<$.001 (0.025)               0.527   
            R Superficial Nuclei   $<$.001 (0.026)         0.050$^{*}$   
            R Laterobasal Nuclei     0.013 (0.033)               0.697   
Hippocampus L Anterior               0.009 (0.023)               0.693   
            L Intermediate           0.023 (0.020)               0.241   
            L Posterior            $<$.001 (0.022)               0.540   
            R Anterior             $<$.001 (0.027)               0.704   
            R Intermediate           0.016 (0.020)               0.438   
            R Posterior              0.046 (0.021)         0.032$^{*}$   
Insula      L Dorsal Anterior        0.019 (0.026)               0.466   
            L Posterior            $<$.001 (0.024)               0.264   
            L Ventral Anterior       0.009 (0.028)               0.751   
            R Dorsal Anterior        0.047 (0.024)         0.049$^{*}$   
            R Posterior            $<$.001 (0.024)               0.282   
            R Ventral Anterior       0.015 (0.026)               0.565   
Striatum    L Caudal (Dorsal)      $<$.001 (0.030)               0.218   
            L Caudal (Ventral)     $<$.001 (0.033)         0.024$^{*}$   
            L Dorsolateral           0.006 (0.031)               0.836   
            L Dorsal               $<$.001 (0.033)               0.055   
            L Rostral                0.004 (0.029)               0.894   
            L Ventral              $<$.001 (0.030)               0.760   
            R Caudal (Dorsal)      $<$.001 (0.031)               0.123   
            R Caudal (Ventral)     $<$.001 (0.031)         0.040$^{*}$   
            R Dorsolateral         $<$.001 (0.033)               0.884   
            R Dorsal               $<$.001 (0.038)         0.002$^{*}$   
            R Rostral                0.001 (0.030)               0.972   
            R Ventral              $<$.001 (0.031)               0.422   
vmPFC       R sgACC                  0.042 (0.024)               0.085   
            Anterior Medial          0.034 (0.027)               0.207   
            Central Dorsal           0.032 (0.024)               0.176   
            Posterior                0.050 (0.024)         0.035$^{*}$   
            Central Ventral          0.026 (0.023)               0.247   
            L sgACC                  0.022 (0.025)               0.389   

                                  FDR-corrected p-value 95\% CI L 95\% CI H  
ROI         Seed                                                             
Amygdala    L Centromedial Nuclei                 0.927    -0.050     0.045  
            L Superficial Nuclei                  0.927    -0.094     0.080  
            L Laterobasal Nuclei                  0.927    -0.082     0.033  
            R Centromedial Nuclei                 0.697    -0.065     0.033  
            R Superficial Nuclei                  0.149    -0.102    -0.000  
            R Laterobasal Nuclei                  0.697    -0.052     0.078  
Hippocampus L Anterior                            0.693    -0.036     0.054  
            L Intermediate                        0.693    -0.015     0.062  
            L Posterior                           0.693    -0.056     0.029  
            R Anterior                            0.704    -0.064     0.043  
            R Intermediate                        0.658    -0.024     0.055  
            R Posterior                           0.097     0.004     0.087  
Insula      L Dorsal Anterior                     0.699    -0.032     0.069  
            L Posterior                           0.69